In [ ]:
from deap import base, creator, gp

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.renameArguments(ARG0='x')

creator.create("Individual", gp.PrimitiveTree)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

expr = toolbox.individual()
nodes, edges, labels = gp.graph(expr)

### Graphviz Section ###
import pygraphviz as pgv

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")

In [1]:
import sys
sys.path.append('/home/cfusting/Dropbox/arctic-browning')
sys.path.append('/home/cfusting/gp_mecl')

In [2]:
import importlib
EXPERMINENT_NAME = "penguin"
experiment = importlib.import_module("experiments." + EXPERMINENT_NAME)

In [6]:
from utilities import lib
TRAINING = "/home/cfusting/design_matrices/training_matrix.hdf"
predictors, response = lib.get_predictors_and_response(TRAINING)
lst_days, snow_days = lib.get_lst_and_snow_days(TRAINING)
pset = experiment.get_pset(predictors.shape[1], lst_days, snow_days)

In [13]:
import cachetools
from deap import creator, base
from gp.algorithms import afpo
creator.create("ErrorSizeComplexity", base.Fitness, weights=(-1.0, -1.0, -1.0))
validate_toolbox = experiment.get_validation_toolbox(predictors, response, pset,
                                                     size_measure=afpo.evaluate_fitness_size_complexity,
                                                     expression_dict=cachetools.LRUCache(maxsize=100),
                                                     fitness_class=creator.ErrorSizeComplexity)

In [36]:
import glob
RESULTS_PATH = "/home/cfusting/symbolic_results"
SEEDS = ['17166', '17785', '246', '25960', '26777', '29676', '423', '8761']
from ndvi import gp_processing_tools
pareto_files = glob.glob(RESULTS_PATH + "/pareto_*_po_{}_*.log".format(EXPERMINENT_NAME))
filtered_files = []
for s in SEEDS:
    filtered_files.extend([x for x in pareto_files if s in x])
all_inds = dict()
for f in filtered_files:
    last_front = gp_processing_tools.get_last_pareto_front(f)
    all_inds[f] = []
    for ind_tuple in last_front:
        tree_string = ind_tuple[1]
        pareto_ind = creator.Individual.from_string(tree_string, pset)
        pareto_ind = creator.Individual(pareto_ind)
        all_inds[f].append(pareto_ind)

/home/cfusting/symbolic_results/pareto_afsc_po_penguin_17166.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_17785.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_246.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_25960.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_26777.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_29676.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_423.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_8761.log


In [35]:
from deap import gp
import pygraphviz as pgv
i = 0
for key in all_inds.keys():
    for ind in all_inds[key]
        nodes, edges, labels = gp.graph(ind)
        g = pgv.AGraph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        g.layout(prog="dot")
        for i in nodes:
            n = g.get_node(i)
            n.attr["label"] = labels[i]
        g.draw("trees/" + key + "_" + i + ".eps")
        i += 1

IndexError: list index out of range